In [ ]:
import numpy as np
import time
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct, WhiteKernel,
                                              ConstantKernel)
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import normalize
import plotly
import plotly.plotly as py
import matplotlib as mpl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly.offline.offline import _plot_html
plotly.offline.init_notebook_mode()
np.random.seed(1)


***SKLEARN TEST EXAMPLE***


In [ ]:
def f(x):
    """The function to predict."""
    return x * np.sin(x)

# ----------------------------------------------------------------------
#  First the noiseless case
X_2 = np.atleast_2d([1., 3., 5., 6., 7., 8.]).T

# Observations
y_2 = f(X_2).ravel()

# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
x_2 = np.atleast_2d(np.linspace(0, 10, 1000)).T

# print 'X', X
# print 'y', y
# print 'x', x

# Instanciate a Gaussian Process model
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5)

# Fit to data using Maximum Likelihood Estimation of the parameters
gp.fit(X_2, y_2)
y_2_pred, y_2_cov = gp.predict(x_2, return_cov=True)



***BADLANDS CRATER EXAMPLE***

In [ ]:
file = np.loadtxt("Examples/etopo/e_liklsurf_s_true/exp_data.txt", delimiter = ',')
data = file[:,0:2].reshape(file.shape[0], 2)
y = np.asarray(file[:,2]).reshape(file.shape[0],1)
# y = normalize(y, axis = 0, norm = 'max') # 'l1, l2, max'
y = y.ravel()
y = y/(-1*y.min())
# y = y*(-1)
y_min = y.min()
y_max = y.max()
X = np.asarray(data)

In [ ]:
def SurrogateHeatmap(x1Data, x2Data, yData, y_min, y_max, title):
    trace = go.Heatmap(x=x1Data, y=x2Data, z=yData, zmin = y_min, zmax = y_max)
    data = [trace]
    layout = Layout(
        title='%s ' %(title),
        scene=Scene(
            zaxis=ZAxis(title = 'Likl',range=[-1,0] ,gridcolor='rgb(255, 255, 255)',gridwidth=2,zerolinecolor='rgb(255, 255, 255)',zerolinewidth=2),
            xaxis=XAxis(title = 'Rain', range=[x2Data.min(),x2Data.max()],gridcolor='rgb(255, 255, 255)',gridwidth=2,zerolinecolor='rgb(255, 255, 255)',zerolinewidth=2),
            yaxis=YAxis(title = 'Erod', range=[x2Data.min(),x2Data.max()] ,gridcolor='rgb(255, 255, 255)',gridwidth=2,zerolinecolor='rgb(255, 255, 255)',zerolinewidth=2),
            bgcolor="rgb(244, 244, 248)"
        )
    )
    fig = Figure(data=data, layout=layout)
    plotly.offline.plot(fig, auto_open=False, output_type='file', filename='Examples/etopo/e_liklsurf_s_true/GP_skl/%s.html' %(title), validate=False)
    return

In [ ]:
SurrogateHeatmap(X[:,0], X[:,1], y, y_min, y_max, 'data')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#print 'Y\n', y 
print y.shape
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

In [ ]:
# kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3))+ WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
kernel = 0.00316**2 + Matern(length_scale=1.11, nu=1.5) + WhiteKernel(noise_level=0.0912)
# kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) + Matern(length_scale=1.11, nu=1.5) + WhiteKernel(noise_level=0.099)

In [ ]:
start = time.time()
# First run
gp = GaussianProcessRegressor(kernel=kernel,alpha=0.0, n_restarts_optimizer = 5, normalize_y = True)

gp.fit(X_train, y_train)
y_mean, y_cov = gp.predict(X_test ,return_cov=True)

end = time.time()
print 'time elapsed in GP ', end-start

In [ ]:
print "Mean squared error: %.2f"% mean_squared_error(y_test, y_mean)
# Explained variance score: 1 is perfect prediction
print 'Variance score: %.2f' % r2_score(y_test, y_mean)

In [ ]:
SurrogateHeatmap(X_test[:,0], X_test[:,1], y_test, y_min, y_max, 'test_data')

In [ ]:
SurrogateHeatmap(X_test[:,0], X_test[:,1], y_mean, y_min, y_max,'GP_pred')

In [ ]:
kernels = [1.0 * RBF(length_scale=10.0, length_scale_bounds=(1e-2, 1e3))+ WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1)),
           1.0 * RationalQuadratic(length_scale=100.0, alpha=0.5),
           1.0 * ExpSineSquared(length_scale=1.0, periodicity=3.0,length_scale_bounds=(0.1, 10.0),periodicity_bounds=(1.0, 10.0)),
           ConstantKernel(0.1, (0.01, 10.0))* (DotProduct(sigma_0=1.0, sigma_0_bounds=(0.0, 10.0)) ** 2),
           0.00316**2 + Matern(length_scale=1.11, nu=1.5) + WhiteKernel(noise_level=0.0912)]
for fig_index, kernel in enumerate(kernels):
    # Specify Gaussian Process
    gp_t = GaussianProcessRegressor(kernel=kernel,alpha=10.0, normalize_y = True)

    # Plot prior
    plt.figure(fig_index, figsize=(8, 8))
    plt.subplot(2, 1, 1)
    X_ = np.linspace(0, 5, 100)
    y_mean, y_std = gp_t.predict(X_[:, np.newaxis], return_std=True)
    plt.plot(X_, y_mean, 'k', lw=3, zorder=9)
    plt.fill_between(X_, y_mean - y_std, y_mean + y_std,
                     alpha=0.2, color='k')
    y_samples = gp_t.sample_y(X_[:, np.newaxis], 10)
    plt.plot(X_, y_samples, lw=1)
    plt.xlim(0, 5)
    plt.ylim(-3, 3)
    plt.title("Prior (kernel:  %s)" % kernel, fontsize=12)

    # Generate data and fit GP
    gp_t.fit(X_train, y_train)
    y_mean_t, y_std_t = gp_t.predict(X_test ,return_cov=True)
    print 'kernel number - 0', fig_index
    print "Mean squared error: %.2f"% mean_squared_error(y_test, y_mean_t)
    print 'Variance score: %.2f' % r2_score(y_test, y_mean_t)
    SurrogateHeatmap(X_test[:,0], X_test[:,1], y_mean_t, y_min, y_max,'GP_pred_%s'%fig_index)